In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
# from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
def load_ucf_101(data_dir, num_frames=16):
    video_paths = []
    labels = []

    dataset=[]
    row={}

    activity_classes = os.listdir(data_dir)
    # print(len(activity_classes))

    for idx, activity_class in enumerate(activity_classes):
        # print(idx)
        class_path = os.path.join(data_dir+'/', activity_class)

        video_paths.append(class_path)
        labels.append(activity_class[2:-12])

        row={
            'path':class_path,
            'labels':activity_class[2:-12]
        }

        dataset.append(row)

        # for video_file in video_files:
        #     video_path = os.path.join(class_path, video_file)
        #     video_paths.append(video_path)
        #     labels.append(idx)

    return dataset

In [ ]:
data_directory = "/content/drive/MyDrive/ManualDataset"
dataset = load_ucf_101(data_directory)

In [ ]:
import pandas as pd
df=pd.DataFrame(dataset)
df.to_csv('Dataset for HAV.csv',index=False)
# dataset

In [ ]:
# df=pd.read_csv('Dataset for HAV.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0], df.iloc[:, 1], test_size=0.2, random_state=42)

In [ ]:
X_train_df = pd.DataFrame(X_train, columns=['path'])
y_train_df = pd.DataFrame(y_train, columns=['labels'])

train_df = pd.concat([X_train_df, y_train_df], axis=1)

print(train_df)

                                                  path        labels
97   /content/drive/MyDrive/ManualDataset/v_Typing_...        Typing
227  /content/drive/MyDrive/ManualDataset/v_Basketb...    Basketball
692  /content/drive/MyDrive/ManualDataset/v_Mopping...  MoppingFloor
587  /content/drive/MyDrive/ManualDataset/v_Typing_...        Typing
884  /content/drive/MyDrive/ManualDataset/v_BabyCra...  BabyCrawling
..                                                 ...           ...
106  /content/drive/MyDrive/ManualDataset/v_YoYo_g1...          YoYo
270  /content/drive/MyDrive/ManualDataset/v_Punch_g...         Punch
860  /content/drive/MyDrive/ManualDataset/v_BabyCra...  BabyCrawling
435  /content/drive/MyDrive/ManualDataset/v_YoYo_g1...          YoYo
102  /content/drive/MyDrive/ManualDataset/v_YoYo_g0...          YoYo

[739 rows x 2 columns]


In [ ]:
train_df.head()

,path,labels
97,/content/drive/MyDrive/ManualDataset/v_Typing_...,Typing
227,/content/drive/MyDrive/ManualDataset/v_Basketb...,Basketball
692,/content/drive/MyDrive/ManualDataset/v_Mopping...,MoppingFloor
587,/content/drive/MyDrive/ManualDataset/v_Typing_...,Typing
884,/content/drive/MyDrive/ManualDataset/v_BabyCra...,BabyCrawling


In [ ]:
X_test_df = pd.DataFrame(X_test, columns=['path'])
y_test_df = pd.DataFrame(y_test, columns=['labels'])

test_df = pd.concat([X_test_df, y_test_df], axis=1)


In [ ]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 3s 0us/step


In [ ]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(df["labels"]))
print(label_processor.get_vocabulary())

labels = train_df["labels"].values
labels = label_processor(labels[..., None]).numpy()
labels

['BabyCrawling', 'Basketball', 'FloorGymnastics', 'MoppingFloor', 'Punch', 'Typing', 'YoYo']


array([[5],
       [1],
       [3],
       [5],
       [0],
       [5],
       [0],
       [3],
       [5],
       [3],
       [4],
       [4],
       [6],
       [3],
       [0],
       [6],
       [4],
       [4],
       [5],
       [5],
       [6],
       [4],
       [3],
       [4],
       [1],
       [6],
       [6],
       [4],
       [6],
       [5],
       [4],
       [0],
       [1],
       [1],
       [2],
       [0],
       [4],
       [4],
       [3],
       [3],
       [2],
       [6],
       [3],
       [3],
       [6],
       [5],
       [3],
       [5],
       [0],
       [5],
       [5],
       [0],
       [3],
       [3],
       [1],
       [4],
       [6],
       [6],
       [1],
       [6],
       [4],
       [2],
       [0],
       [4],
       [6],
       [6],
       [6],
       [6],
       [2],
       [2],
       [0],
       [4],
       [2],
       [0],
       [2],
       [0],
       [2],
       [6],
       [4],
       [0],
       [0],
       [4],
       [3],
    

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["path"].values.tolist()

    labels = df["labels"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()


    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 29ms/step


In [ ]:
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

Frame features in train set: (739, 20, 2048)
Frame masks in train set: (739, 20)
train_labels in train set: (739, 1)
test_labels in train set: (185, 1)


In [ ]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30

def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
17/17 [==============================] - ETA: 0s - loss: 1.8452 - accuracy: 0.2534
Epoch 1: val_loss improved from inf to 1.59309, saving model to ./tmp/video_classifier
17/17 [==============================] - 14s 195ms/step - loss: 1.8452 - accuracy: 0.2534 - val_loss: 1.5931 - val_accuracy: 0.4189
Epoch 2/30
17/17 [==============================] - ETA: 0s - loss: 1.6319 - accuracy: 0.3501
Epoch 2: val_loss improved from 1.59309 to 1.52899, saving model to ./tmp/video_classifier
17/17 [==============================] - 0s 23ms/step - loss: 1.6319 - accuracy: 0.3501 - val_loss: 1.5290 - val_accuracy: 0.4505
Epoch 3/30
16/17 [===========================>..] - ETA: 0s - loss: 1.4921 - accuracy: 0.4336
Epoch 3: val_loss improved from 1.52899 to 1.35280, saving model to ./tmp/video_classifier
17/17 [==============================] - 0s 30ms/step - loss: 1.4926 - accuracy: 0.4313 - val_loss: 1.3528 - val_accuracy: 0.5991
Epoch 4/30
15/17 [=========================>....] - ETA: 

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    # for i in np.argsort(probabilities)[::-1]:
    #     print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    print(f'{class_vocab[np.argmax(probabilities)]}')

    return frames

test_video = np.random.choice(test_df["path"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: /content/drive/MyDrive/ManualDataset/v_Typing_g16_c03.avi
1/1 [==============================] - 0s 24ms/step
Typing


In [ ]:
sequence_model.save('final_model.h5')